# AIS DECODER

In [ ]:
#aisdecoder env
import pandas as pd
import geopandas as gpd
from decoder import NMEA_decoder
from datetime import date
from shapely.geometry import Point, Polygon
import folium 
import numpy as np



from mapper import *

today = date.today().strftime("%Y-%m-%d")

style = 'symbol:pin; fill:#0000ff; fill-opacity:0.7; stroke:#ffffff; stroke-opacity:1.0; stroke-width:0.5'
text = None
dateTime = None

In [ ]:
file_path = "SRC\\Input\\CSK_21_01_2.txt"
ais_msg = NMEA_decoder(file_path)
GPS = ais_msg.get_GPS()
ais = ais_msg.to_df()
# ais.to_excel('SRC\\Output\\131021\\Ais_decoded.xlsx')
# GPS.to_excel('SRC\\Output\\131021\\GPS_decoded.xlsx')
ais.dropna(axis=0,subset=['lon','lat'], inplace=True)
ais.reset_index(inplace=True, drop=True)

# MAPPER


In [ ]:
style = 'symbol:pin; fill:#0000ff; fill-opacity:0.7; stroke:#ffffff; stroke-opacity:1.0; stroke-width:0.5'
text = None
dateTime = None

# output_path = 'SRC\\Output\\'
# decoded_path = '071021'

# ais = pd.read_excel(output_path+decoded_path+'\\'+'Ais_decoded.xlsx')
# ais.drop(ais.filter(regex='Unname'), axis=1, inplace=True)
# ais=ais[ais['lon'].notnull()]
# ais.reset_index(drop=True, inplace=True)

# GPS = pd.read_excel(output_path+decoded_path+'\\'+'GPS_decoded.xlsx')
# GPS.drop(GPS.filter(regex='Unname'), axis=1, inplace=True)

In [ ]:
ais = ais[ (ais['timestamp']>'16:57:10') & (ais['timestamp']<'16:59:50')]
# ais = ais.drop_duplicates(subset='mmsi')
ais.dropna(axis=0,subset=['lon','lat'], inplace=True)
ais.reset_index(inplace=True, drop=True)

# Create Shapefiles

In [ ]:
# SHP = gpd.GeoDataFrame(columns = ['style_css', 'label', 'text', 'dateTime', 'geometry'])
# # ais = ais[ (ais['timestamp']>'16:44:59') & (ais['timestamp']<'16:59:24')]
# ais.reset_index(inplace=True, drop=True)

# output_path = 'SRC\\Output\\'
# decoded_path = '131021'

# for i in range(len(ais)):
#     label = ais['mmsi'][i]
#     # lat, lon = ais['latitude'][i], ais['longitude'][i]
#     lat, lon = float(ais['lat'][i]), float(ais['lon'][i])
#     coordinates = [(lon, lat)]
#     geometry = Point(coordinates)
#     new_row = {'style_css':style, 'label':label, 'text':text, 'dateTime':dateTime, 'geometry':geometry}
#     SHP = SHP.append(new_row, ignore_index=True)
# SHP.to_file(output_path+decoded_path+'\\'+'navi.shp')

### GPS

In [ ]:
GPS.head(3)

In [ ]:
SHP = gpd.GeoDataFrame(columns = ['style_css', 'label', 'text', 'dateTime', 'geometry'])
GPS = GPS[ (GPS['timestamp']>'16:56:00') & (GPS['timestamp']<'16:57:00')]
GPS.reset_index(inplace=True, drop=True)

output_path = 'SRC\\Output\\'
# decoded_path = '131021'

for i in range(len(GPS)):
    label = GPS['timestamp'][i]
    lat, lon = GPS['latitude'][i], GPS['longitude'][i]
    coordinates = [(lon, lat)]
    geometry = Point(coordinates)
    new_row = {'style_css':style, 'label':label, 'text':text, 'dateTime':dateTime, 'geometry':geometry}
    SHP = SHP.append(new_row, ignore_index=True)
SHP.to_file(output_path+'GPS.shp')

In [ ]:
SHP

# Create HTML

In [ ]:
ships_mmsi = ais['mmsi'].unique() # mmsi of ships identified
n_ships_mmsi = ais['mmsi'].nunique() # number of ships identified


for mmsi in ships_mmsi:
     PlotPolyline(m, SubsetAis_mmsi(mmsi, ais))

# plotReceiver(GPS.iloc[0:1], m)
points = np.arange(0,GPS.shape[0], 100) # Sampling 20 sec

for point in points:
     plotReceiver(GPS.iloc[point:point+1], m)


add_my_legend(m)
m
# df = SubsetAis_mmsi(ships_mmsi[3], ais)
# points = [(x.lat, x.lon) for x in df.iloc]
# points

In [ ]:
m.save('map.html')